In [1]:
# Install library yang diperlukan 

import warnings; warnings.simplefilter('ignore')
!pip install unidecode
!pip install textblob
!pip install sastrawi
!pip install tqdm
!pip install wget

In [2]:
# Import library yang diperlukan 
import numpy as np
import pandas as pd
import re
from unidecode import unidecode

from html import unescape
from tqdm import tqdm
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# modul impor file
import urllib.request
import wget


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dkd091122\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# URL dataset di GitHub

url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/Duolingo.csv'
#url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/subset_dataset1.csv'
# Membaca dataset langsung dari URL
df = pd.read_csv(url, low_memory=False, encoding='utf8')
df

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,0,8c03a08a-285c-4e67-a36a-831f579ffbe6,Pu Put,https://play-lh.googleusercontent.com/a/ACg8oc...,aplikasinya bagusss buat belajar bahasa asingg...,5,56,5.139.5,2024-02-21 14:46:17,NaN,NaN,5.139.5
1,1,27635f36-15ed-4c48-9067-c8ce257aa342,Sri Hidayati,https://play-lh.googleusercontent.com/a-/ALV-U...,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5,42,5.137.5,2024-02-05 13:56:41,NaN,NaN,5.137.5
2,2,c03bddcf-ccf4-4307-822d-a55caf84cb47,Stephanie Artha,https://play-lh.googleusercontent.com/a/ACg8oc...,とても good.. aku bisa belajar bahasa Inggris dan...,5,1265,5.139.5,2024-02-17 21:29:06,NaN,NaN,5.139.5
3,3,a86f481c-fa0a-4310-aeef-fb72a71dc344,Linda Saumi (Linda),https://play-lh.googleusercontent.com/a-/ALV-U...,"Aplikasi yang seru, mudah dipahami, ada rank s...",1,453,5.138.5,2024-02-12 20:24:32,NaN,NaN,5.138.5
4,4,4945e054-e64c-4fbb-bc03-6a440e4a4bfe,Silvi Maulidini,https://play-lh.googleusercontent.com/a-/ALV-U...,"aplikasinya seru, kayak game jadi kita gak bos...",5,47,5.137.6,2024-02-15 21:17:20,NaN,NaN,5.137.6
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,ac6676dd-e5e5-4cb9-b92e-a41c1feee3ab,Yuli Yusnita,https://play-lh.googleusercontent.com/a/ACg8oc...,"Sangat cocok dan mudah teruntuk saya,saya suka...",5,0,5.112.4,2023-07-24 16:24:18,NaN,NaN,5.112.4
19996,19996,77f3a897-efaa-4292-b499-fde19b2b2894,Desi Harianti,https://play-lh.googleusercontent.com/a-/ALV-U...,App Duolingo sangat bagus segera di download s...,5,0,4.68.5,2020-06-22 11:18:45,NaN,NaN,4.68.5
19997,19997,55bbfc6a-d258-4dbf-bfbc-15cad2604e60,Clarista Sianturi,https://play-lh.googleusercontent.com/a/ACg8oc...,Gamenya seru dan bisa buat belajar walau lumay...,5,0,5.109.4,2023-07-12 07:23:09,NaN,NaN,5.109.4
19998,19998,e3f67ab6-4c19-4ddb-9634-e03aa7c6dd68,ara beby,https://play-lh.googleusercontent.com/a-/ALV-U...,Game gajelas nanti tulisannya berubah ubah con...,3,0,5.83.4,2022-12-05 20:37:58,NaN,NaN,5.83.4


In [4]:
score_counts = df['score'].value_counts().sort_index()
score_counts

score
1      651
2      388
3      800
4     2189
5    15972
Name: count, dtype: int64

In [5]:
df = df[['content', 'score']]
df

,content,score
0,aplikasinya bagusss buat belajar bahasa asingg...,5
1,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5
2,とても good.. aku bisa belajar bahasa Inggris dan...,5
3,"Aplikasi yang seru, mudah dipahami, ada rank s...",1
4,"aplikasinya seru, kayak game jadi kita gak bos...",5
...,...,...
19995,"Sangat cocok dan mudah teruntuk saya,saya suka...",5
19996,App Duolingo sangat bagus segera di download s...,5
19997,Gamenya seru dan bisa buat belajar walau lumay...,5
19998,Game gajelas nanti tulisannya berubah ubah con...,3


In [6]:
df[df.duplicated()==True]

,content,score
2780,sangat bagus untuk belajar berbagai bahasa,5
3131,sangat membantu belajar bahasa asing,5
3197,sangat membantu untuk belajar bahasa asing,5
3250,sangat bagus untuk belajar berbagai bahasa,5
3266,sangat membantu dalam belajar bahasa,5
...,...,...
19315,sangat membantu dan bermanfaat,5
19336,Sangat bermanfaat,5
19340,bagus sekali,5
19345,Sangat membantu dalam belajar bahasa,5


In [7]:
# memeriksa duplikasi
print(df.duplicated().sum())

# membuang duplikasi
df.drop_duplicates(inplace=True)
print(df.shape)
     

305
(19695, 2)


In [8]:
df

,content,score
0,aplikasinya bagusss buat belajar bahasa asingg...,5
1,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5
2,とても good.. aku bisa belajar bahasa Inggris dan...,5
3,"Aplikasi yang seru, mudah dipahami, ada rank s...",1
4,"aplikasinya seru, kayak game jadi kita gak bos...",5
...,...,...
19995,"Sangat cocok dan mudah teruntuk saya,saya suka...",5
19996,App Duolingo sangat bagus segera di download s...,5
19997,Gamenya seru dan bisa buat belajar walau lumay...,5
19998,Game gajelas nanti tulisannya berubah ubah con...,3


In [10]:
def pelabelan(score):
    if score < 3 : # ulasan dengan score dibawah 3 diberi nilai negatif
        return 'negatif'
    elif score == 3 : # ulasan dengan score sama dengan 3 diberi nilai netral
        return 'netral'
    elif score > 3 : # ulasan dengan score sama dengan 3 diberi nilai positif
        return 'positif'
   # elif score == 5 : # ulasan dengan score sama dengan 3 diberi nilai positif
       # return 'positif'
df['label'] = df['score'].apply(pelabelan)
df

,content,score,label
0,aplikasinya bagusss buat belajar bahasa asingg...,5,positif
1,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5,positif
2,とても good.. aku bisa belajar bahasa Inggris dan...,5,positif
3,"Aplikasi yang seru, mudah dipahami, ada rank s...",1,negatif
4,"aplikasinya seru, kayak game jadi kita gak bos...",5,positif
...,...,...,...
19995,"Sangat cocok dan mudah teruntuk saya,saya suka...",5,positif
19996,App Duolingo sangat bagus segera di download s...,5,positif
19997,Gamenya seru dan bisa buat belajar walau lumay...,5,positif
19998,Game gajelas nanti tulisannya berubah ubah con...,3,netral


In [11]:
score_counts = df['label'].value_counts().sort_index()
score_counts

label
negatif     1039
netral       800
positif    17856
Name: count, dtype: int64

In [12]:
# modul impor file
import urllib.request
url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/slang.txt'
##https://raw.githubusercontent.com/taudataanalytics/eLearning/master/data/slang.dic##
##https://raw.githubusercontent.com/taudataanalytics/eLearning/master/data/slang.txt##
response = urllib.request.urlopen(url)
slang_id = response.read().decode('utf-8').split('\n')

print("jumlah slang: ", len(slang_id))

jumlah slang:  2280


In [13]:
import requests

slang_id = {}
response = requests.get('https://raw.githubusercontent.com/hafizaah95/DATA/main/slang.txt')
for line in response.text.splitlines():
    key, value = line.strip().split(':')
    slang_id[key] = value
print("jumlah slang : ", len(slang_id))

jumlah slang :  1962


In [14]:
# modifikasi sesuka hati jika dirasa masih ada slang yang belum tertangani
slang_tambahan = {'geblek': 'goblok', 'smoga': 'semoga', 'trimakasih': 'terima kasih', 'jln2': 'jalan-jalan', 'apk': 'aplikasi', 'bljar': 'belajar',
                  'fitur2': 'fitur', 'moga': 'semoga', 'untk': 'untuk', 'ngk': 'tidak', 'app' : 'aplikasi', 'jdi' : 'jadi'}
slang_id.update(slang_tambahan)

print("jumlah slang : ", len(slang_id))

jumlah slang :  1974


In [15]:

url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/stopword.txt'
response = urllib.request.urlopen(url)
stopword_id = response.read().decode('utf-8').split('\n')

print("jumlah stopword: ", len(stopword_id))

jumlah stopword:  1349


In [16]:
# modifikasi sesuka hati jika dirasa masih ada stopword yang belum tertangani
stopword_tambahan = set(['a', 'anak', 'allah', 'acara', 'arti', 'asing', 'anak-anak'
                         'ah', 'aja',
                         'b', 'bikin', 'bang', 'bangsa', 'bilang', 'bicara', 'biar', 'bilang', 'belajar', 'bosan',
                         'c', 'cemungut',
                         'd', 'daerah', 'deh', 'dukung', 'duolingo', 'dungolingo', 'download', 'dalam', 'dll', 'dan', 'dengan'
                         'e', 'eyes',
                         'f', 'face', 'film', 'folded',
                         'g', 
                         'h', 'hand', 'hitam', 'hands',
                         'i', 'indonesia', 'iya','its',
                         'j', 'jakarta', 'jalan', 'jaring', 'jadi'
                         'k', 'khusus', 'kepala', 'keluarga', 'kah', 'kerja', 'kosakata',
                         'l', 'lihat', 'laki',
                         'm', 'mata', 'mbak', 'malam', 'muda', 'mas',
                         'n', 'name', 'nonton', 'nya', 'nomor',
                         'p', 'provider', 'putih', 'pilih', 'pakai', 'pimpin', 'product', 'program', 'pas'
                         'q',
                         'r', 'rumah',
                         's', 'smilling', 'smiling', 'si', 'sih', 'suara', 'sinyal', 'sandi', 'salam', 'saya'
                         't', 'tunggu', 'teman', 'tonton', 'the', 'tuju', 'tears', 'thank', 'terimakasih'
                         'u', 'username', 'url',
                         'v', 
                         'w', 'with',
                         'x',
                         'y', 'yang', 'you'
                         'z',
                         '4g'])
stopword_id = set(stopword_id).union(stopword_tambahan)

print("jumlah stopword: ", len(stopword_id))

jumlah stopword:  1455


In [17]:

def cleaning_content(content):
    # menghapus url
    url_pattern = re.compile(r'(\w+:\/\/\S+)')
    content = url_pattern.sub(' ', content)
    # menghapus hashtag
    hashtag_pattern = re.compile(r'#\w+\b')
    content = hashtag_pattern.sub(' ', content)
    # menghapus nama pengguna (memuat "@")
    username_pattern = re.compile(r'@\w+\b\s*')
    content = username_pattern.sub(' ', content)
    # menghapus angka
    content = re.sub(r'\b\d+\b|[^\w\s]', '', content)
    # menghapus huruf yang berulang
    content = re.sub(r'(\w)(\1{2,})', r'\1', content)
    # Menghapus karakter Jepang
    content = re.sub(r'[\u3040-\u309F\u30A0-\u30FF]+', '', content)
    # menghapus karakter yang tidak diperlukan
    content = re.sub(r'\n', ' ', content)
    content = re.sub(r'\r', ' ', content)
    content = re.sub(r'&\w+', '', content)
    content = re.sub(r'\s+', ' ', content).strip()
    # Whitespace stripping
    content = content.strip()
    # menangani huruf kapital dan spasi
    content = unidecode(unescape(content.lower()))
    return content

def remove_stopword(content):
    word_tokens = word_tokenize(content)
    filtered_sentence = [w for w in word_tokens if not w in stopword_id]
    return ' '.join(filtered_sentence)

def stemming_and_lemmatization(content):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(content)

def remove_slang(content):
    content = ''.join(slang_id[p] if p in slang_id else p for p in re.split(r'(\W+)', content))
    return content

def tanda_baca(content):
    # mengahpus simbol
    symbol_pattern = re.compile(r'[^\w\s]+')
    content = symbol_pattern.sub(' ', content)
    return content
    
def remove_repeated_words(content):
    words = content.split()
    seen = set()
    result = []
    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)
    return ' '.join(result)
    
def preprocess(content):
    # cleaning text and lowercase
    output = cleaning_content(content)
    # remove slang
    output = remove_slang(output)
    # stemming and lemmatization
    output = stemming_and_lemmatization(output)
    # remove stopwords
    output = remove_stopword(output)
    #menghapus tanda baca
    output = tanda_baca(content)
    #menghapus kata yang berulang
    output = remove_repeated_words(output)
    
    return output

     

In [18]:
df['clean_content'] = ''
for idx, post in tqdm(df.iterrows()):
  df.at[idx, 'clean_content'] = preprocess(post.content)

19695it [3:21:31,  1.63it/s]


In [19]:
df = df[['clean_content', 'score', 'label']]
df

,clean_content,score,label
0,aplikasinya bagusss buat belajar bahasa asingg...,5,positif
1,asli Duolingo bagus banget Dari awal malas bel...,5,positif
2,とても good aku bisa belajar bahasa Inggris dan J...,5,positif
3,Aplikasi yang seru mudah dipahami ada rank set...,1,negatif
4,aplikasinya seru kayak game jadi kita gak bose...,5,positif
...,...,...,...
19995,Sangat cocok dan mudah teruntuk saya suka teri...,5,positif
19996,App Duolingo sangat bagus segera di download s...,5,positif
19997,Gamenya seru dan bisa buat belajar walau lumay...,5,positif
19998,Game gajelas nanti tulisannya berubah ubah con...,3,netral


In [20]:
df.to_csv('Data_Bersih.csv', index = False)